In [46]:
import re
import os
import sys
sys.path.append('..')

from dotenv import load_dotenv
print(load_dotenv(os.path.join('..', '.env')))

from src.llm.llm_client import llm_client
from src.retrieval.encoder_client import embedding_client
from src.prompts.base_prompt import BASE_SYSTEM_PROMPT
from src.prompts.topic_prompt import EXTRACT_TOPIC_SYSTEM_PROMPT
from src.retrieval.embeddings import get_topic, get_embeddings
from src.retrieval.similarity import search_similarity_chunk

True


# Данные для бенчмарка

In [56]:
queries = [
    "Почему нефть подорожала?",
    "Почему цена золота изменилась?",
    "Какой рост акций у Газпрома и почему?",
    "Из-за чего акции банка Тинькоффа выросли?",
    "Что повлияло на бурный рост акций Газпрома?",
    "Почему акции Мвидео в последние дни среагировали ростом?",
    "Какой фактор повлиял на то, что нефть подорожала?", 
    "Почему золото растет последний год?",
    "Почему акции Сбербанка упали в цене?",
    "Что вызвало упадок стабильности курса рубля",
    "Почему Индекс ММВБ в последнее время растут?", 
    "Из-за чего снизилась цена на нефть?",
    "Почему золото стало дешевле в прошлом году?",
    "Что привело к росту акций АФК «Системы»?",
    "Почему Индекс МосБиржи продолжает снижаться?",
    "Из-за чего поднялись акции МТС?",
    "Почему облигации в России стали менее доходными?", 
    "Что повлияло на рост цены на золото?",
    "Почему акции Лукойл показали рост в этом квартале?", 
    "Из-за чего упала цена на нефть?",
    "Почему акции Яндекса выросла за последние месяцы?", 
    "Что вызвало снижение акций Т-Банк?",
    "Почему золото стало дороже в этом году?", 
    "Из-за чего выросли акции Роснефти?",
    "Почему облигации в России стали менее привлекательными?",
    "Что привело к падению цены на золото?",
    "Почему акции АЛРОСА упали в этом месяце?",
    "Из-за чего выросла цена на нефть?",
    "Почему криптовалюта TON показывает снижение?", 
    "Что повлияло на снижение акций Аэрофлота?"
]

In [57]:
true_topics = [
    "нефть",
    "золото", 
    "акции", 
    "акции", 
    "акции", 
    "акции", 
    "нефть", 
    "золото", 
    "акции",
    "другое",
    "акции", 
    "нефть", 
    "золото", 
    "акции",
    "акции",
    "акции", 
    "облигации", 
    "золото", 
    "акции", 
    "нефть",
    "акции", 
    "акции", 
    "золото",
    "акции", 
    "облигации",
    "золото", 
    "акции", 
    "нефть", 
    "криптовалюта", 
    "акции"
]

In [55]:
true_contexts = [
    "Россия объявила о дополнительном сокращении поставок нефти на 200 тысяч баррелей в сутки доведя общий объем сокращения до 500 тысяч баррелей в сутки и продлила это решение до конца первого квартала 2024 года. Дефицит бюджета России в 2023 году может составить около 1% ВВП что лучше ожиданий. На фоне роста ключевой ставки ЦБ РФ банки недополучат 600 млрд рублей процентного дохода. На рынке акций компании Whoosh и КуйбышевАзот объявили о дивидендных выплатах. Фьючерсы на индексы Мосбиржи и РТС демонстрируют рост а на товарно-сырьевом рынке наблюдается снижение цен на нефть Urals и газ при этом Brent и золото растут. Продажи смартфонов в России достигли рекордного уровня с 2020 года чему способствовала маркетинговая активность и восстановление экономики после пандемии.",
    "Госдума рассматривает законопроект о введении дополнительного налога на добычу полезных ископаемых (НДПИ) на золото в размере 78 тыс рублей за килограмм. Налог будет действовать с 1 июня по 31 декабря 2024 года. Это предложение может повлиять на рентабельность золотодобывающих компаний и стоимость золота на рынке.",
    "Инвесторы анализируя комментарии менеджмента «Газпрома» предположили что компания сохраняет возможность выплаты дивидендов по итогам 2023 года. Это связано с ожиданиями стабильной финансовой ситуации и достаточного уровня прибыли для распределения среди акционеров.",
    "Расписки Тинькофф Банка (TCS Group) на Московской бирже выросли на 1% после объявления компании о планах делистинга расписок с Лондонской биржи начиная с 31 января. Этот рост может быть связан с оптимизмом инвесторов относительно концентрации торгов на одной площадке что может упростить управление капиталом и повысить ликвидность акций на Мосбирже.",
    "Инвесторы анализируя комментарии менеджмента «Газпрома» предположили что компания сохраняет возможность выплаты дивидендов по итогам 2023 года. Это связано с ожиданиями стабильной финансовой ситуации и достаточного уровня прибыли для распределения среди акционеров.",
    "Компания М.Видео планирует размещение облигаций на общую сумму от 5 миллиардов рублей. Это решение направлено на привлечение дополнительного финансирования для развития бизнеса возможного расширения сети или рефинансирования существующих долгов. Ренессанс Капитал выступает организатором размещения.",
    "Россия объявила о дополнительном сокращении поставок нефти на 200 тысяч баррелей в сутки доведя общий объем сокращения до 500 тысяч баррелей в сутки и продлила это решение до конца первого квартала 2024 года. Дефицит бюджета России в 2023 году может составить около 1% ВВП что лучше ожиданий. На фоне роста ключевой ставки ЦБ РФ банки недополучат 600 млрд рублей процентного дохода. На рынке акций компании Whoosh и КуйбышевАзот объявили о дивидендных выплатах. Фьючерсы на индексы Мосбиржи и РТС демонстрируют рост а на товарно-сырьевом рынке наблюдается снижение цен на нефть Urals и газ при этом Brent и золото растут. Продажи смартфонов в России достигли рекордного уровня с 2020 года чему способствовала маркетинговая активность и восстановление экономики после пандемии.",
    "Компания «Полюс» планирует к 2030 году увеличить производство золота в два раза — до 6 млн унций в год. Это позволит ей занять второе место в мире по объемам добычи золота. Увеличение производства связано с реализацией новых проектов и расширением существующих мощностей.",
    "Акции Сбербанка снизились на 145% на предыдущей торговой сессии закрывшись на отметке 27925 руб. Это уже третий день подряд снижения что приближает цену к ключевым уровням поддержки. Внешний фон был негативным из-за падения цен на нефть но акции Сбербанка отреагировали сдержанно так как ранее уже просели на фоне санкционных угроз. Инвесторы ожидают решения ЦБ по ключевой ставке которое может повлиять на динамику акций. Долгосрочный тренд остается восходящим несмотря на текущие коррекции.",
    "Российский рубль приблизился к отметке 100 за доллар в начале осени и аналитики прогнозируют отсутствие значительного укрепления валюты в краткосрочной перспективе и даже к 2025 году. Основными факторами давления на рубль являются геополитическая нестабильность санкции снижение экспортных доходов и отток капитала. Возможности для роста курса рубля ограничены так как на его динамику влияют как внешние так и внутренние экономические факторы.",
    "Российские фондовые индексы показали положительную динамику в конце торговой сессии вторника. Индекс ММВБ вырос на 115% а индекс РТС увеличился на 113% благодаря ослаблению рубля что сделало российские активы более привлекательными для иностранных инвесторов.",
    "11-й пакет антироссийских санкций согласован, включая ограничения на 70 физических лиц, 30 юридических лиц и южную ветку нефтепровода Дружба. Санкции направлены на снижение экспортных возможностей России в энергетическом секторе и ограничение доступа к международным рынкам.",
    "С весны 2022 года компания «Полюс» крупнейший производитель золота в России столкнулась с рядом вызовов включая санкционное давление ограничения на экспорт и логистические сложности. Это негативно сказалось на её финансовых показателях и рыночной капитализации что было отмечено аналитиками «Открытие Инвестиции». Компания также столкнулась с ростом затрат и снижением доступности западных технологий.",
    "Четыре актива АФК Системы — Cosmos Hotel Group Медси Биннофарм Групп и Степь — с высокой вероятностью проведут IPO в ближайшие 1-2 года. Это заявление сделал исполнительный директор по связям с инвесторами Дмитрий Коровкин. Выход на IPO может быть связан с планами компании по привлечению капитала для дальнейшего развития и расширения бизнеса.",
    "Индекс МосБиржи начал торги в четверг с небольшим снижением что может быть связано с общей рыночной неопределенностью геополитическими факторами или коррекцией после предыдущих торгов. Такое движение отражает текущие настроения инвесторов на российском рынке акций.",
    "Компания МТС отложила принятие дивидендной политики из-за неопределенности на рынке связанной с экономической ситуацией или внутренними факторами компании. Это решение может повлиять на ожидания инвесторов и стоимость акций компании в краткосрочной перспективе.",
    "На фоне высокой волатильности рынков акций и облигаций вызванной неопределенностью в отношении инфляции и процентных ставок в России инвесторы ищут способы защиты от процентного риска. Банк России сохраняет жесткую риторику возможны повышения ключевой ставки до 20% и выше. В таких условиях флоатеры — облигации с плавающей ставкой купона привязанной к ключевой ставке или RUONIA — становятся эффективным инструментом для защиты капитала. Они предлагают доходность около 193% у надежных эмитентов и более 20% у эмитентов умеренного качества. Альтернативы такие как банковские депозиты и фонды денежного рынка уступают по доходности. Для упрощения инвестиций в флоатеры запущен фонд Тинькофф Пассивный Доход (TPAY) который предлагает ожидаемую доходность 20% годовых.",
    "Новая неделя начинается с роста цен на драгоценные металлы такие как золото и серебро на фоне ослабления индекса доллара. Снижение стоимости доллара делает драгметаллы более привлекательными для инвесторов что стимулирует рост их цен.",
    "Акции компании ЛУКОЙЛ снизились на 13% до 45395 руб. что хуже общего снижения рынка на 083%. Объем торгов составил 56 млрд руб. что значительно выше среднемесячного уровня. Технический анализ указывает на пробой нисходящего треугольника и высокую активность продавцов при этом RSI упал ниже 70% что может замедлить распродажи. Неопределенность с отчетом за 2022 год и дивидендами оказывает давление на котировки. Внешний фон умеренно негативный с падением азиатских индексов и нефти Brent. Долгосрочные перспективы остаются умеренно позитивными несмотря на риски связанные с эмбарго на нефть в ЕС.",
    "Россия объявила о дополнительном сокращении поставок нефти на 200 тысяч баррелей в сутки доведя общий объем сокращения до 500 тысяч баррелей в сутки и продлила это решение до конца первого квартала 2024 года. Дефицит бюджета России в 2023 году может составить около 1% ВВП что лучше ожиданий. На фоне роста ключевой ставки ЦБ РФ банки недополучат 600 млрд рублей процентного дохода. На рынке акций компании Whoosh и КуйбышевАзот объявили о дивидендных выплатах. Фьючерсы на индексы Мосбиржи и РТС демонстрируют рост а на товарно-сырьевом рынке наблюдается снижение цен на нефть Urals и газ при этом Brent и золото растут. Продажи смартфонов в России достигли рекордного уровня с 2020 года чему способствовала маркетинговая активность и восстановление экономики после пандемии.",
    "Аналитики оценивают справедливую стоимость акций Яндекса (YNDX) в 5055 рублей прогнозируя потенциал роста на 96%. Компания признана фаворитом среди источников РДВ на 2024 год благодаря сильным позициям на рынке и перспективам роста.",
    "Рассматривается возможность введения санкций против крупных российских банков таких как Альфа-банк Росбанк и Тинькофф Банк. Это может быть связано с геополитической напряженностью или действиями которые противоречат международным нормам. Санкции могут включать ограничения на доступ к международным финансовым системам и заморозку активов.",
    "Цена золота в рублях превысила отметку 264 000 рублей за тройскую унцию, что связано с ростом спроса на золото как на защитный актив на фоне экономической нестабильности и ослабления рубля. Также на стоимость золота влияют глобальные факторы, такие как инфляция и изменения в монетарной политике крупных центральных банков.",
    "Акции Роснефти выросли на 263% в последнюю торговую сессию недели достигнув цены закрытия 614 рублей. Объем торгов составил 78 млрд рублей. На фоне умеренно позитивного внешнего фона включая рост фьючерсов на индекс S&P 500 и нефть Brent акции Роснефти показали лучшую динамику по сравнению с рынком. Технический анализ указывает на продолжение роста с возможностью коррекции к уровням поддержки 570–578 рублей. Долгосрочные цели роста котировок находятся в диапазоне 620–650 рублей.",
    "На фоне высокой волатильности рынков акций и облигаций вызванной неопределенностью в отношении инфляции и процентных ставок в России инвесторы ищут способы защиты от процентного риска. Банк России сохраняет жесткую риторику возможны повышения ключевой ставки до 20% и выше. В таких условиях флоатеры — облигации с плавающей ставкой купона привязанной к ключевой ставке или RUONIA — становятся эффективным инструментом для защиты капитала. Они предлагают доходность около 193% у надежных эмитентов и более 20% у эмитентов умеренного качества. Альтернативы такие как банковские депозиты и фонды денежного рынка уступают по доходности. Для упрощения инвестиций в флоатеры запущен фонд Тинькофф Пассивный Доход (TPAY) который предлагает ожидаемую доходность 20% годовых.",
    "Госдума рассматривает законопроект о введении дополнительного налога на добычу полезных ископаемых (НДПИ) на золото в размере 78 тыс рублей за килограмм. Налог будет действовать с 1 июня по 31 декабря 2024 года. Это предложение может повлиять на рентабельность золотодобывающих компаний и стоимость золота на рынке.",
    "Акции АЛРОСА снизились на 015% на предыдущей торговой сессии закрывшись на уровне 667 руб. что хуже общего роста рынка на 128%. Бумаги консолидируются под уровнем 68 руб. демонстрируя слабую динамику из-за отсутствия драйверов для роста после отказа от выплаты дивидендов за 2021 год. В краткосрочной перспективе возможны риски обновления минимумов с вероятностью падения ниже 65 руб. что может привести к снижению до 56 руб. В долгосрочной перспективе акции остаются под давлением несмотря на потенциал восстановления до уровней 115–120 руб. при улучшении рыночных условий. Внешний фон поддерживает позитивные настроения на азиатских рынках а нефть Brent торгуется около $119.",
    "К открытию торговой сессии 13 сентября индекс ЖиС составил 41, что указывает на спокойствие на рынках. Курс рубля к юаню — 12.63, а цена нефти — 72.2 доллара за баррель. Добыча нефти странами ОПЕК+ в августе 2024 года достигла 34.56 млн баррелей в сутки. Минэкономразвития прогнозирует рост цен на российский газ для Китая. Ключевые события дня",
    "После ареста Павла Дурова, ключевой фигуры экосистемы TON, криптовалюта упала на 18%. Telegram, который активно поддерживает TON, продвигая проекты и упрощая использование блокчейна через мессенджер, столкнулся с рисками из-за возможного усиления прозрачности для властей, что может привести к оттоку пользователей и отключению поддержки блокчейна. Это ставит под вопрос будущее TON, так как вера в экосистему во многом связана с личностью Дурова. В случае реализации негативного сценария TON может упасть до $2, тогда как благополучное разрешение ситуации может быстро восстановить позиции криптовалюты.",
    "Акции Аэрофлота упали на 403% на предыдущей торговой сессии закрывшись на уровне 2668 руб. Несмотря на падение акции торговались лучше рынка с объемом торгов в 345 млн руб. Технический анализ указывает на возможный отскок к уровню 278–282 руб. но при пробое зоны сопротивления 285–30 руб. возможен рост к цели 3018 руб. Долгосрочный импульс роста начавшийся в ноябре 2020 года был сломан и среднесрочные цели падения находятся в диапазоне 217–272 руб. Внешний фон остается умеренно негативным с падением фьючерсов на S&P 500 и азиатских индексов а также снижением цен на нефть Brent."
]

# LLM-as-a-Judge

In [59]:
SYSTEM_PROMPT_JUDGE = """Ты - AI-судья по оценке других LLM.
Ты оцениваешь LLM на согласованность генерации ответа относительно предоставленного контекста.
Выдай бинарную оценку: "1" или "0", где:
- Оценка "1" - ставится, если модель ответила по контексту и не использовала своих знаний.
- Оценка "0" - ставится, если модель ответила не по контексту и выдала выдуманный ответ, несогласованный с контекстом.

В ответе выдай строго только бинарную оценку, без лишних слов, формулировок и текста.
Данные предоставленны ниже:"""

# Прогон бенчмарка

In [60]:
score_topic = 0.
score_retrieval = 0.
score_generate = 0.
answers = []


for idx, user in enumerate(queries):
    
    ### Извлечение топика и проверка на согласованность
    topic = get_topic(user)
    score_topic += (topic == true_topics[idx])

    ### Проверка retrieval-части
    embedding = get_embeddings(user)
    filters_kwargs = {"topic": topic}
    chunks = search_similarity_chunk(embedding, filters_kwargs=filters_kwargs)[0]
    score_retrieval += (true_contexts[idx] in chunks)

    ### Проверка этапа генерации
    context = "\n\n".join([f"КОНТЕКСТ №{idx}:\n{chunk}" for idx, chunk in enumerate(chunks, 1)])
    messages = [
        {"role": "system", "content": BASE_SYSTEM_PROMPT.format(context=context)},
        {"role": "user", "content": user}
    ]
    answer = llm_client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False
    ).choices[0].message.content
    answers.append(answer)

    message_judge = [
        {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
        {"role": "user", "content": f"###КОНТЕКСТ:\n{context}\n\nОТВЕТ LLM ДЛЯ ОЦЕНКИ:\n{answer}"}
    ]

    answer_judge = llm_client.chat.completions.create(
        model="deepseek-chat",
        messages=message_judge,
        stream=False
    ).choices[0].message.content
    score_generate += int(answer_judge)

    print(score_topic)
    print(score_retrieval)
    print(score_generate)
    print('=' * 100)

1.0
1.0
1.0
2.0
2.0
2.0
3.0
2.0
2.0
4.0
2.0
3.0
5.0
2.0
3.0
6.0
2.0
3.0
7.0
3.0
4.0
8.0
4.0
5.0
9.0
4.0
5.0
10.0
4.0
5.0
11.0
4.0
5.0
12.0
5.0
5.0
13.0
6.0
6.0
14.0
6.0
7.0
15.0
6.0
7.0
16.0
6.0
7.0
17.0
7.0
8.0
18.0
8.0
9.0
19.0
8.0
9.0
20.0
9.0
9.0
21.0
9.0
9.0
22.0
9.0
9.0
23.0
10.0
10.0
24.0
10.0
10.0
25.0
11.0
11.0
26.0
12.0
12.0
27.0
12.0
12.0
28.0
13.0
12.0
29.0
14.0
13.0
30.0
14.0
13.0


In [62]:
n = 30

print(f"ACC TOPIC: {round(score_topic / n, 2)}")
print(f"ACC RETRIEVAL: {round(score_retrieval / n, 2)}")
print(f"ACC GENERATE: {round(score_generate / n, 2)}")

ACC TOPIC: 1.0
ACC RETRIEVAL: 0.47
ACC GENERATE: 0.43
